### Run AutoQuake
#### Directory structure
##### SAC data (sac_parent_dir / date / *.SAC)
```
SAC/ -> sac_parent_dir     
└── 20240401/ -> date      
    ├── TW.station.00.HHZ.D.2024,093,20:22:12.SAC     
    └── TW.station.00.HHZ.D.2024,093,22:35:17.SAC            
```
##### HDF5 (h5_parent_dir / date / time_window.h5)
```
Hualien_0403_MiDAS/ -> h5_parent_dir     
└── 20240401/ -> date       
    ├── MiDAS_20240403_80700_81000.h5 -> time_window.h5     
    └── MiDAS_20240403_8100_8400.h5 
```
##### PZ file (pz_dir / *.pz)
```
pz_dir/ -> pz_dir     
├── SAC_PZs_TW_ZUZH_HH1_00_2019.001.00.00.00.0000_2599.365.23.59.59.99999
└── SAC_PZs_TW_ZUZH_HNZ_10_2019.001.00.00.00.0000_2599.365.23.59.59.99999
```
Date would acquire througth the reading of the gamma_events.csv from GaMMA.  
The reason why we need date is to automatically find the sac data due to we might have the events across the day.

### Scenarios (Seiemomter only, ~~DAS only, Both~~)
#### Seismometer only


##### 1. Define the **Path** and some important parameters for later use.

In [1]:
from pathlib import Path

your_repository_path = Path('/home/patrick/Work/AutoQuake')
result_path = your_repository_path / 'result_test'
sac_parent_dir = Path('/raid1/share/for_patrick/AutoQuake_testset/SAC') # this is the testset, u can try it.
startdate = '20240402'
enddate = '20240402' # using same startdate and enddate for single day test.
station = Path('/raid1/share/for_patrick/AutoQuake_testset/station_seis.csv')
pz_dir = Path('/raid1/share/for_patrick/AutoQuake_testset/PZ_dir/')
model_3d = your_repository_path / 'H3DD' / 'tomops_H14'
gamma_vel_model = Path('/raid1/share/for_patrick/AutoQuake_testset/vel_model/Hualien_1D.vel')

##### 2. Create the object for each step.       
       
> **⚠️ Important:** Please check the detailed argument in the class if u want further manipulation, or you can just put your mouse on the PhaseNet object to see its docstring supported by VScode.

-  PhaseNet    

In [3]:
from autoquake.picker import PhaseNet

phasenet = PhaseNet(
    data_parent_dir=sac_parent_dir,
    start_ymd=startdate,
    end_ymd=enddate,
    result_path=result_path,
    format='SAC',
    model='phasenet',
    device='cpu',
)

-  GaMMA   

In [4]:
from autoquake.associator import GaMMA

gamma = GaMMA(
    station=station,
    result_path=result_path,
    pickings=phasenet.picks, # we use the attribute from PhaseNet object, make sure PhaseNet object has been created.
    vel_model=gamma_vel_model,
)

-  H3DD    

In [4]:
from autoquake.relocator import H3DD

h3dd = H3DD(
    gamma_event=gamma.events, # gamma attribute.
    gamma_picks=gamma.picks, # gamma attribute.
    station=gamma.station, # gamma attribute.
    model_3d=model_3d,
    event_name=result_path.name, 
)

cp from /home/patrick/Work/AutoQuake/H3DD/tomops_H14 to /data2/patrick/AutoQuake/H3DD


cp: ‘/home/patrick/Work/AutoQuake/H3DD/tomops_H14’ and ‘/data2/patrick/AutoQuake/H3DD/tomops_H14’ are the same file


##### 3-1. Run it through default scenario: AutoQuake       
> **⚠️ Important:** Since the important parameter setting only on **PhaseNet**, **GaMMA**, and **H3DD**, you can just put the object of these three steps into the run_autoquake function, and providing a **PZ_dir** contains the pz file. The **magnitude**, **polarity**, also **GAfocal** would use the result from these steps.

In [5]:
from autoquake.scenarios import run_autoquake
run_autoquake(
    picker=phasenet,
    associator=gamma,
    relocator=h3dd,
    pz_dir=pz_dir,
    use_polarity=True, # default as True
    use_magnitude=True, # default as True
    mag_processes=30, # default as 30, processes for multiprocessing
    use_focal=True, # default as True
    type_judge=None # Using None as default, which means using the second character of station to judge the type of station.
)

Not using distributed mode
Total samples:  /raid1/share/for_patrick/AutoQuake_testset/SAC/20240402/.SAC : 39 files


Predicting:   0%|          | 0/39 [00:00<?, ?it/s]

/raid1/share/for_patrick/AutoQuake_testset/SAC/20240402/TW.ESA.10.HN*


Predicting:   3%|▎         | 1/39 [00:10<06:26, 10.16s/it]

/raid1/share/for_patrick/AutoQuake_testset/SAC/20240402/TW.FUSS.00.HH*


...

psbasemap: Constructing basemap
pstext: Plotted 1 text-strings
psmeca: Number of records read: 1


 Pfile : 240402213653.P24        
  0.818181813    
 # of solutions:           1
240402213653.P24


psbasemap: Constructing basemap
pstext: Plotted 1 text-strings
psmeca: Number of records read: 1


 Pfile : 240402235810.P24        
   1.00000000    
 # of solutions:           1
240402235810.P24


psbasemap: Constructing basemap
pstext: Plotted 1 text-strings
psmeca: Number of records read: 1


4. Go through the result folder to check the result.

- Phasenet

In [5]:
import pandas as pd
df = pd.read_csv(result_path / 'picks_phasenet' / startdate / 'picks.csv')
df.head(3)

,station_id,phase_index,phase_time,phase_score,phase_type,dt_s,phase_amplitude
0,TW.WHY.10.HN,12219,2024-04-02T00:02:02.190,0.356,S,0.01,1.072
1,TW.SSLB..HH,14380,2024-04-02T00:02:23.808,0.371,P,0.01,6736.000
2,TW.SSLB..HH,14461,2024-04-02T00:02:24.618,0.520,S,0.01,6736.000


- GaMMA

In [8]:
df_event = pd.read_csv(result_path / 'gamma_events.csv')
df_event[df_event['event_index'] == 24] # 0403 main eqrthquake

,time,magnitude,sigma_time,sigma_amp,cov_time_amp,gamma_score,num_picks,num_p_picks,num_s_picks,event_index,x(km),y(km),z(km),longitude,latitude,depth_km
12,2024-04-02T23:58:09.773,999,0.512,0,0,23.996,23,21,2,24,21.78,0.224,21.091,121.582,23.858,21.091


In [9]:
df_picks = pd.read_csv(gamma.picks)
df_picks[df_picks['event_index'] == 24]

,station_id,phase_index,phase_time,phase_score,phase_type,dt_s,phase_amplitude,event_index,gamma_score
5123,SM06,8629344,2024-04-02T23:58:14.200000,0.853,P,0.01,73940.00,24,0.272607
5124,SHUL,8629457,2024-04-02T23:58:14.570000,0.364,P,0.01,6551000.00,24,0.100939
5125,HWA,8629467,2024-04-02T23:58:14.670000,0.443,P,0.01,5223000.00,24,0.141951
5126,SM02,8629361,2024-04-02T23:58:15.010000,0.813,P,0.01,162500.00,24,0.258484
5127,SM39,8629509,2024-04-02T23:58:15.180000,0.794,P,0.01,233100.00,24,0.261715
5128,SM09,8629465,2024-04-02T23:58:15.260000,0.736,P,0.01,144700.00,24,0.240031
5129,NACB,8629703,2024-04-02T23:58:17.038000,0.567,P,0.01,35380000.00,24,0.183315
5130,SM39,8629798,2024-04-02T23:58:18.070000,0.436,S,0.01,793900.00,24,0.099376
5131,SM02,8629670,2024-04-02T23:58:18.100000,0.332,S,0.01,407300.00,24,0.108741
5132,EHYH,8629854,2024-04-02T23:58:18.540000,0.555,P,0.01,264300.00,24,0.164483


- H3DD

In [5]:
from autoquake.visualization._plot_base import check_format
df_h3dd, _ = check_format(result_path/f'{result_path.name}.dat_ch.hout')
df_h3dd.iloc[-1]

latitude                         23.876
longitude                       121.552
depth_km                          20.01
time         2024-04-02T23:58:10.320000
Name: 60, dtype: object

> **⚠️ Important:**  Once you understand the flow, please use python script to run it, jupyter notebook might crash due to the high memory loading.

- polarity

#### DAS

In [1]:
from pathlib import Path
result_path = Path('/home/patrick/Work/AutoQuake/testset_DAS')
h5_parent_dir = Path("/raid1/share/for_patrick/AutoQuake_testset/hdf5/")
startdate = '20240401'
enddate = '20240403'
station = Path('/home/patrick/Work/EQNet/tests/hualien_0403/station_das.csv')
model_3d = Path('/home/patrick/Work/AutoQuake/H3DD/tomops_H14')
gamma_vel_model_1 = Path('/home/patrick/Work/AutoQuake/vel_model/midas_vel.csv')
gamma_vel_model_2 = Path(
    '/home/patrick/Work/AutoQuake_pamicoding/GaMMA/Hualien_data_20240402/Hualien_1D.vel'
)

In [2]:
from autoquake.picker import PhaseNet

phasenet = PhaseNet(
    data_parent_dir=h5_parent_dir,
    start_ymd=startdate,
    end_ymd=enddate,
    result_path=result_path,
    format='h5',
    model='phasenet_das',
    device='cpu',
)
phasenet.run_predict()

Not using distributed modeNot using distributed mode
Not using distributed mode

Total samples: 0 files
Total samples: 0 files


Predicting: 0it [00:00, ?it/s]
Predicting: 0it [00:00, ?it/s]


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte